# Preconditioner Comparison

## Packages

In [ ]:
import torch
import gpytorch
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
import gpboost as gpb
import requests
import pandas as pd
import time

## Function to generate data

In [ ]:
# Download notebook from GitHub
url = 'https://raw.githubusercontent.com/TimGyger/FSVA/refs/heads/main/Simulation/Simulate_Data.py'

# Download the Python file
response = requests.get(url)

# Save the Python file locally
with open('your_script.py', 'wb') as f:
    f.write(response.content)

print("Python file downloaded successfully!")

# Execute the downloaded Python file using exec()
with open('your_script.py', 'r') as f:
    code = f.read()

exec(code)

## Non-Gaussian Data

In [1]:
#X, y, b = simulate_gp_response("bernoulli-logit",100000, 0, 1.0, torch.tensor([(0.25, 0.50,0.75,1.00,1.25)]),1)

In [2]:
# Convert X and y to a DataFrame
# data = pd.DataFrame(X.numpy(), columns=[f"x{i+1}" for i in range(X.shape[1])])
# data['y'] = y.numpy()
# data['b'] = b.numpy()

# Save to CSV
# data.to_csv("simulated_data_non_Gaussian.csv", index=False)
# print("Data saved to 'simulated_data_non_Gaussian.csv'")

In [ ]:
# URL of the raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/TimGyger/FSVA/refs/heads/main/Simulation/simulated_data_non_Gaussian.csv'

# Load the CSV file directly from the URL
df = pd.read_csv(url)

# Select the first 5 columns for X
X = df.iloc[:, :5]  # First 5 columns

# Select the last column for y
y = df.iloc[:, 5]  # Last column

In [ ]:
# Convert to numpy:
X_np = X.to_numpy()
y_np = y.to_numpy()

## Experiments


In [ ]:
# Different number of sample vectors
vector_sample = [10, 20, 50, 100]
# Number of repetitions
num_rep = 100
# Zero matrices
matrix = np.zeros((len(vector_ind_points), num_rep))
matrix_t = np.zeros((len(vector_ind_points), num_rep))

matrix1 = np.zeros((len(vector_ind_points), num_rep))
matrix_t1 = np.zeros((len(vector_ind_points), num_rep))
# Nested loop to iterate over both vectors
for i, val1 in enumerate(vector_sample):
    for j in range(0, num_rep):
        # Start the timer
        start_time = time.time()
        # FSVecchia with euclidean-based neighbor search
        model_fsva = gpb.GPModel(gp_coords=X_np, cov_function="gaussian_ard", 
                                 likelihood="bernoulli_logit",num_neighbors = 30,num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "iterative", gp_approx="vecchia",seed = 2)
        model_fsva.set_optim_params(params={"cg_preconditioner_type": "predictive_process_plus_diagonal",
                                            "piv_chol_rank": 200, "seed_rand_vec_trace": j+1, "num_rand_vec_trace": val1})
        neg_fsva = model_fsva.neg_log_likelihood(cov_pars = np.array([1.0, 0.25, 0.50,0.75,1.00,1.25]), y = y_np)
        matrix[i, j] = neg_fsva
        # End the timer
        end_time = time.time()

        # Calculate elapsed time in seconds
        matrix_t[i, j] = end_time - start_time

        start_time = time.time()
        # FSVecchia with euclidean-based neighbor search
        model_fsva = gpb.GPModel(gp_coords=X_np, cov_function="gaussian_ard", 
                                 likelihood="bernoulli_logit",num_neighbors = 30,num_ind_points = 200,ind_points_selection = "kmeans++",
                                 matrix_inversion_method = "iterative", gp_approx="vecchia",seed = 2)
        model_fsva.set_optim_params(params={"cg_preconditioner_type": "Bt_Sigma_inv_plus_W_B",
                                            "seed_rand_vec_trace": j+1, "num_rand_vec_trace": val1})
        neg_fsva = model_fsva.neg_log_likelihood(cov_pars = np.array([1.0, 0.25, 0.50,0.75,1.00,1.25]), y = y_np)
        matrix1[i, j] = neg_fsva
        # End the timer
        end_time = time.time()

        # Calculate elapsed time in seconds
        matrix_t1[i, j] = end_time - start_time

## Plots

In [ ]:
# P1: Log-likelihood Box Plot
plt.figure(figsize=(8, 6))

# Group the data by 't' and plot box plots
plt.boxplot(matrix.transpose(), widths=0.3, patch_artist=True,
            medianprops=dict(color='black'))
plt.boxplot(matrix1.transpose(), widths=0.3, patch_artist=True, 
            positions=np.array(range(1, len(matrix1)+1)) + 0.5,  # Shifting the second box plot for clarity
            boxprops=dict(facecolor='lightblue', color='black'),
            medianprops=dict(color='black'))  # Add matrix1 data in light blue
plt.ylabel("Log-likelihood")
plt.xticks([])
plt.xlabel("")
plt.tight_layout()
plt.show()

# P2: Time Plot
plt.figure(figsize=(8, 6))

row_means = np.mean(matrix_t, axis=1)
plt.plot(vector_ind_points, row_means, linewidth=1.5)
row_means_t1 = np.mean(matrix_t1, axis=1)
plt.plot(vector_ind_points, row_means_t1, linewidth=1.5, label='FSVADU', color='green')
plt.ylabel("Time (s)")
plt.xlabel("Number of inducing points")
plt.legend(title="")
plt.tight_layout()
#plt.annotate(f"Cholesky: {time_chol} s", xy=(0.02, 0.95), xycoords='axes fraction', 
#             fontsize=12, fontweight='bold', ha='left', va='top', bbox=dict(boxstyle="round", facecolor="white", edgecolor="black"))
plt.show()